In [1]:
import src
import numpy as np
from src.learner import ICaRLLearner
from box import Box

In [2]:
def _get_instance(module, config, *args):

    cls = getattr(module, config.name)
    return cls(*args, **config.get('kwargs', {}))

In [8]:
config = Box.from_yaml(filename='/home/tony/Incremental_Learning/configs/train/icarl_config.yaml')
saved_dir = Path(config.main.saved_dir)
if not saved_dir.is_dir():
saved_dir.mkdir(parents=True)

logging.info(f'Save the config to "{saved_dir}".')
copyfile(args.config_path, saved_dir / 'config.yaml')

if not args.test:

random_seed = config.main.get('random_seed')
if random_seed is None:
    torch.backends.cudnn.deterministic = False
    torch.backends.cudnn.benchmark = True
else:
    logging.info('Make the experiment results deterministic.')
    random.seed(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

logging.info('Create the device.')
if 'cuda' in config.trainer.kwargs.device and not torch.cuda.is_available():
    raise ValueError("The cuda is not available. Please set the device to 'cpu'.")
device = torch.device(config.trainer.kwargs.device)

logging.info('Create the network architecture.')
net = _get_instance(src.model.nets, config.net).to(device)

logging.info('Create the loss functions and corresponding weights.')
loss_fns, loss_weights = [], []
defaulted_loss_fns = [loss_fn for loss_fn in dir(torch.nn) if 'Loss' in loss_fn]
for config_loss in config.losses:
    if config_loss.name in defaulted_loss_fns:
        loss_fn = _get_instance(torch.nn, config_loss)
    else:
        loss_fn = _get_instance(src.model.losses, config_loss)
    loss_fns.append(loss_fn)
    loss_weights.append(config_loss.weight)

logging.info('Create the metric functions.')
metric_fns = MetricFns()
for config_metric in config.metrics:
    metric_fn = _get_instance(src.model.metrics, config_metric).to(device)
    name = config_metric.get('alias', config_metric.name)
    setattr(metric_fns, name, metric_fn)

yes


In [12]:
a = list(range(5, 10))
b = list(range(0, 5))
for ai, bi in zip(a, b):
    print(f'{ai}, {bi}')

5, 0
6, 1
7, 2
8, 3
9, 4


In [5]:
loaded_path

'None'

In [3]:
config = Box.from_yaml(filename='/home/tony/Incremental_Learning/configs/train/icarl_config.yaml')
config.learner.setdefault('kwargs', {}).update(config=config)
learner = _get_instance(src.learner, config.learner)

NameError: name 'config' is not defined

In [6]:
learner.current_task += 1
dataset = learner.build_dataset()
loader = learner.build_dataloader(dataset, 'train')
print(len(loader))
count = np.zeros(100)
for data in loader:
    for i in range(10):
        gt = data['target'][i, 0, 0]
        count[gt] += 1
print(count)
loader = learner.build_dataloader(dataset, 'valid')
print(len(loader))
count = np.zeros(100)
for data in loader:
    for i in range(10):
        gt = data['target'][i, 0, 0]
        count[gt] += 1
print(count)

900
[  0.   0.   0.   0. 450.   0.   0.   0.   0.   0.   0. 450.   0.   0.
   0.   0.   0.   0. 450.   0.   0.   0.   0.   0. 450.   0.   0.   0.
   0. 450.   0.   0. 450.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0. 450.   0. 450.   0.   0. 450.   0.   0.   0.   0.   0.
   0. 450.   0.   0.   0.   0.   0.   0.   0.   0. 450. 450.   0. 450.
   0.   0.   0.   0.   0.   0.   0. 450.   0. 450.   0.   0.   0. 450.
 450.   0.   0.   0. 450.   0.   0.   0.   0. 450.   0.   0. 450.   0.
   0.   0.]
100
[ 0.  0.  0.  0. 50.  0.  0.  0.  0.  0.  0. 50.  0.  0.  0.  0.  0.  0.
 50.  0.  0.  0.  0.  0. 50.  0.  0.  0.  0. 50.  0.  0. 50.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0. 50.  0. 50.  0.  0. 50.  0.  0.  0.
  0.  0.  0. 50.  0.  0.  0.  0.  0.  0.  0.  0. 50. 50.  0. 50.  0.  0.
  0.  0.  0.  0.  0. 50.  0. 50.  0.  0.  0. 50. 50.  0.  0.  0. 50.  0.
  0.  0.  0. 50.  0.  0. 50.  0.  0.  0.]


In [7]:
loader = learner.build_dataloader(dataset, 'test')
print(len(loader))
count = np.zeros(100)
for data in loader:
    for i in range(10):
        gt = data['target'][i, 0, 0]
        count[gt] += 1
print(count)

400
[100.   0.   0.   0. 100.   0.   0.   0. 100.   0. 100. 100.   0.   0.
 100. 100.   0.   0. 100.   0.   0.   0.   0.   0. 100.   0.   0.   0.
   0. 100.   0.   0. 100.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. 100. 100.   0. 100.   0. 100. 100. 100. 100.   0.   0.   0.
   0. 100. 100.   0.   0. 100.   0.   0.   0.   0. 100. 100. 100. 100.
   0.   0. 100.   0.   0.   0.   0. 100. 100. 100.   0.   0.   0. 100.
 100.   0. 100. 100. 100.   0.   0. 100. 100. 100. 100.   0. 100. 100.
   0.   0.]
